In [18]:
# Importing necessary packages
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from PIL import Image
import pytesseract
import cv2
import os
import imutils
import numpy as np
%matplotlib inline

In [2]:
# Checking if GPU available
physical_devices = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [10]:
# Function for getting text from images
def get_text(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    gray_t = cv2.threshold(gray, 0, 255,
                             cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    gray_b = cv2.medianBlur(gray, 3)

    # Writing the grayscale image to disk as a temporary file so we can
    # apply OCR to it
    filename = "Gray/{}.png".format(os.getpid())
    cv2.imwrite(filename, gray_t)

    # load the image as a PIL/Pillow image, apply OCR, and then delete
    # the temporary file
    text_t = pytesseract.image_to_string(Image.open(filename))
    os.remove(filename)
    
    filename = "Gray/{}.png".format(os.getpid())
    cv2.imwrite(filename, gray_b)

    # load the image as a PIL/Pillow image, apply OCR, and then delete
    # the temporary file
    text_b = pytesseract.image_to_string(Image.open(filename))
    os.remove(filename)
    
    text = text_t + ' ' + text_b
    text = ' '.join(text.split())
    return text

In [11]:
# Loading saved model (created in simple_nn.ipynb and saved in drive)
def create_model():
    model = Sequential([
        Embedding(10000, 64),  # 10000: size of vocabulary; 64: output dimension of embedding.
        GlobalAveragePooling1D(),  
        Dense(16, activation = 'relu'),  # For inputting 16 dimension word embeddings
        Dense(1, activation = 'sigmoid') # For outputting either as positive or negative
    ])

    model.compile(
        loss = 'binary_crossentropy', 
        optimizer = 'adam',
        metrics = ['accuracy']
    )

    model.summary()
    return model

model = create_model()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          640000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 641,057
Trainable params: 641,057
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Loads the weights
model.load_weights('cp.ckpt')

In [16]:
# Word_index dict for converting text to encoding
word_index = imdb.get_word_index() 

# Padding the samples
def pad_sample(y): 
    y = [y,]
    y = pad_sequences(y, value = word_index['the'], padding = 'post', maxlen = 256, truncating='post')
    y = y[0]
    for i in range(len(y)):
        if y[i]>10000:
            y[i] = 1
    return y
    
# Converting string text to encoding for model input
def convert_to_encoding(s): 
    l = s.split()
    for i in range(len(l)):
        if l[i] in word_index:
            l[i] = word_index[l[i]]
        else:
            l[i] = word_index['the']
    l = pad_sample(l)        
    return l

In [14]:
file = pd.read_csv('Test.csv')

# Converting category column to string
file['Category'] = file['Category'].astype(str)
file.head()

,Filename,Category
0,Test1001.jpg,nan
1,Test1012.jpg,nan
2,Test1022.jpg,nan
3,Test1071.jpg,nan
4,Test1122.jpg,nan


In [22]:
# Making predictions
j = 0
for i, row in file.iterrows():
    j+=1
    print(j)
    img_name = row[0]
    img_path = '../../Data Files/Dataset/'+img_name
    image = cv2.imread(img_path)
    quote = get_text(image)
    if len(quote)<5:
        file.at[i, 'Category'] = 'Random'
    else:
        temp = convert_to_encoding(quote)
        p = model.predict(np.expand_dims(temp, axis=0))
#         print(p)
        if p>=0.5:
            file.at[i, 'Category'] = 'Positive'
        else:
            file.at[i, 'Category'] = 'Negative'
#     print(j, quote)
#     if j==5:
#         break
        
file.head()        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239


,Filename,Category
0,Test1001.jpg,Negative
1,Test1012.jpg,Negative
2,Test1022.jpg,Positive
3,Test1071.jpg,Negative
4,Test1122.jpg,Positive


In [23]:
print(file['Category'].value_counts())
file.head()

Random      103
Positive     96
Negative     40
Name: Category, dtype: int64


,Filename,Category
0,Test1001.jpg,Negative
1,Test1012.jpg,Negative
2,Test1022.jpg,Positive
3,Test1071.jpg,Negative
4,Test1122.jpg,Positive


In [24]:
file.to_csv('Test.csv')

In [ ]:
# Performance
# Score
# 47.86517